In [151]:
# This is a terminal command 
!pip install pyarrow 

import requests 
from bs4 import BeautifulSoup

import os
from datetime import date

import pandas as pd
import numpy as np

pd.options.display.max_colwidth = 100

URI = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
EXTENSIONS = ['parquet', 'csv', 'json', 'pdf', 'zip']
DATA_DIR = './data'
DIM_FILE = 'dim_scrape.csv'

HASH = 'hash'


In [ ]:
# class ResponceError(Exception):
#     pass

In [128]:
def dim_scrapper():
    with requests.get(URI) as response:
        try: 
            response.raise_for_status()
        except Exception as e:
            raise e
            
        data = response.text

    links = BeautifulSoup(data, 'html.parser').find_all('a', href=True)

    df = pd.DataFrame({
        'uri': [link['href'].strip() if not link['href'].endswith('pdf') else 'https://www.nyc.gov'+link['href'].strip() for link in links]
    })
    
    df = pd.concat(
        [df[df['uri'].str.endswith(ext)] for ext in EXTENSIONS]
    ).reset_index(drop=True).drop_duplicates()

    # enrichemnt  
    df[HASH] = pd.util.hash_pandas_object(df)

    # enrichemnt - loading processing - # TODO: move to the loader function
    df['add_at'] = int(date.today().strftime('%Y%m%d'))
    df['file'] = df['uri'].str.split('/').str[-1]
    df.info()

    df['category'] = np.nan
    df.loc[df['uri'].str.endswith('parquet'), 'category'] = df['file'].str.split('_').str[0]

    df['date'] = np.nan
    df.loc[df['uri'].str.endswith('parquet'),'date'] = df['file'].str.split('_').str[-1]
    df.loc[df['uri'].str.endswith('parquet'),'date'] = df['date'].str.split('.').str[0]

    df['downloaded'] = False
     
    path = os.path.join(DATA_DIR, DIM_FILE)
    if not os.path.exists(path):
        df.to_csv(
            path, 
            index=False,
            mode = 'tw',
        )
    else:
        old = pd.read_csv(path)
        df = pd.concat([
            old, df
        ]).drop_duplicates(
            [HASH]
        ).sort_values(
            'add_at'
        )
        df.to_csv(
            path, index=False,
            mode = 'ta',
            
        )
    
    # return df


In [144]:
def download_file(uri, chunk_size=8192):
    name = uri.split('/')[-1]
    path = os.path.join(DATA_DIR, name)

    try:
        with requests.get(uri, stream=True) as response:       
            with open(path, 'wb') as file:
                for chunk in response.iter_content(chunk_size=40960):
                    file.write(chunk)

        return os.path.isfile(path) and os.path.getsize(path) > 0
    except Exception as e:
        print(f'Unable to download file {uri} due: {e}')
        return False

In [145]:
df = dim_scrapper()

uri = 'https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2021-03.parquet'
download_file(uri, chunk_size=8192)


True

In [136]:
# file =  os.path.join(DATA_DIR, 'yellow_tripdata_2023-04.parquet')

# df = pd.read_parquet(file)
# df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3288250 entries, 0 to 3288249
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [137]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-04-01 00:14:49,2023-04-01 00:45:01,2.0,4.90,1.0,N,48,223,1,28.9,3.5,0.5,6.0,0.00,1.0,39.9,2.5,0.00
1,2,2023-04-01 00:00:24,2023-04-01 00:56:19,1.0,21.89,2.0,N,132,43,2,70.0,0.0,0.5,0.0,6.55,1.0,81.8,2.5,1.25
2,1,2023-04-01 00:03:50,2023-04-01 00:14:42,2.0,1.30,1.0,N,148,113,1,11.4,3.5,0.5,2.0,0.00,1.0,18.4,2.5,0.00
3,1,2023-04-01 00:53:18,2023-04-01 01:01:28,1.0,1.50,1.0,N,249,79,1,10.0,3.5,0.5,1.0,0.00,1.0,16.0,2.5,0.00
4,2,2023-04-01 00:07:00,2023-04-01 00:17:16,2.0,1.49,1.0,N,158,246,1,11.4,1.0,0.5,1.0,0.00,1.0,17.4,2.5,0.00


In [138]:
# csv
# xlsx
# json
# parquet: binary file that can save metadata about the file

In [140]:
# _ = pd.read_csv(os.path.join(DATA_DIR, DIM_FILE))
# _.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492 entries, 0 to 491
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   uri         492 non-null    object
 1   hash        492 non-null    uint64
 2   add_at      492 non-null    int64 
 3   file        492 non-null    object
 4   category    484 non-null    object
 5   date        484 non-null    object
 6   downloaded  492 non-null    bool  
dtypes: bool(1), int64(1), object(4), uint64(1)
memory usage: 23.7+ KB


In [141]:
# _['date'] = pd.to_datetime(_['date'])
# _.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492 entries, 0 to 491
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   uri         492 non-null    object        
 1   hash        492 non-null    uint64        
 2   add_at      492 non-null    int64         
 3   file        492 non-null    object        
 4   category    484 non-null    object        
 5   date        484 non-null    datetime64[ns]
 6   downloaded  492 non-null    bool          
dtypes: bool(1), datetime64[ns](1), int64(1), object(3), uint64(1)
memory usage: 23.7+ KB
